# parcels regions

runs parcels on existing netcdf files

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import utils
from parcels_utils import HFRGrid
from parcels_sim import ParcelsSimulation
from parcels_analysis import ParticlePlotFeature

## configuration

change the contents of `configs` for the simulation configuration you want

In [ ]:
configs = [
    "plume_track.json",
#     "plume_track_totsdlj.json",
#     "tijuana_interped.json",
#     "tijuana_lin.json",
#     "tijuana_lin_aggr.json",
#     "tijuana_less.json",
#     "tijuana_now.json",
#     "tijuana_onerep.json",
#     "tijuana_range.json",
#     "buoy_track.json",
]

domain = None
# domain = dict(
#     S=32.53,
#     N=32.564,
#     W=-117.162,
#     E=-117.105
# )

loaded_configs = [utils.load_config(utils.PARCELS_CONFIGS_DIR / path) for path in configs]
grids = [HFRGrid(utils.FILES_ROOT / cfg["netcdf_path"]) for cfg in loaded_configs]
sims = []
for i in range(len(loaded_configs)):
    sims.append(ParcelsSimulation(loaded_configs[i]["name"], grids[i], loaded_configs[i]["parcels_config"]))

In [ ]:
fs = grids[0].fieldset

## Animated gif stuff and particle simulation

### ParticleSet and spawn point setup

note about interpolation methods: only `linear` works if you want to use the FieldSet in a ParticleSet.

In [ ]:
[sim.pset.show(field="vector", vmax=ParcelsSimulation.MAX_V) for sim in sims]

### simulation setup and execution

simulation parameter setup

In [ ]:
for sim in sims:
    sim.execute()

### gif generation

don't have to run, requires [magick](https://imagemagick.org/index.php)

the gifs will be saved `snapshots/`

In [ ]:
sim.parcels_result.plot_features = []
sim.parcels_result.add_plot_feature(ParticlePlotFeature.get_sd_stations(), station=True)
for sim in sims:
    frames = sim.parcels_result.generate_all_plots(utils.FILES_ROOT / utils.PICUTRE_DIR / sim.name, domain=domain)
    print(sim.parcels_result.generate_gif(frames, utils.FILES_ROOT / utils.PICUTRE_DIR / f"{sim.name}.gif"))